In [2]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

# Day 16

In [26]:
from __future__ import annotations
from typing import *
import re
import itertools
import math

<IPython.core.display.Javascript object>

## Part One

In [4]:
RAW = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

<IPython.core.display.Javascript object>

In [5]:
Range = Tuple[int, int]


def make_range(rng: str) -> Range:
    lo, hi = rng.split("-")
    return (int(lo), int(hi))

<IPython.core.display.Javascript object>

In [6]:
class Rule(NamedTuple):
    name: str
    ranges: Tuple[Range]

    def is_valid(self, x: int) -> bool:
        return any(lo <= x <= hi for lo, hi in self.ranges)

    @staticmethod
    def parse(line: str) -> Rule:
        line = line.strip()
        name, ranges = line.split(": ")
        ranges = ranges.split(" or ")
        ranges = tuple([make_range(rng) for rng in ranges])
        return Rule(name, ranges)

<IPython.core.display.Javascript object>

In [7]:
Ticket = List[int]


def make_ticket(s: str) -> Ticket:
    ticket = [int(x) for x in s.split(",")]
    return ticket

<IPython.core.display.Javascript object>

In [8]:
class Problem(NamedTuple):
    rules: List[Rule]
    my_ticket: Ticket
    nearby_tickets: List[Ticket]

    def is_valid_for_some_field(self, x: int) -> bool:
        return any(rule.is_valid(x) for rule in self.rules)

    def is_invalid_ticket(self, ticket: Ticket) -> bool:
        for val in ticket:
            if not self.is_valid_for_some_field(val):
                return True
        return False

    def get_error_rate(self) -> int:
        error_rate = 0
        for nearby_ticket in self.nearby_tickets:
            for val in nearby_ticket:
                if not self.is_valid_for_some_field(val):
                    error_rate += val
        return error_rate

    def discard_invalid_tickets(self) -> Problem:
        valid_tickets = [
            ticket
            for ticket in self.nearby_tickets
            if not self.is_invalid_ticket(ticket)
        ]
        return self._replace(nearby_tickets=valid_tickets)

    @staticmethod
    def parse(raw: str) -> Problem:
        rules, ticket, nearbys = raw.split("\n\n")
        rules = rules.split("\n")
        rules = [Rule.parse(line) for line in rules]
        ticket = ticket.split("\n")[1]
        ticket = make_ticket(ticket)
        nearbys = nearbys.split("\n")[1:]
        nearbys = [make_ticket(ticket) for ticket in nearbys]
        return Problem(
            rules=rules,
            my_ticket=ticket,
            nearby_tickets=nearbys,
        )

<IPython.core.display.Javascript object>

In [9]:
PROBLEM = Problem.parse(RAW)
assert PROBLEM.get_error_rate() == 71

<IPython.core.display.Javascript object>

In [10]:
with open("../input/day16.txt") as f:
    raw = f.read().strip()
problem = Problem.parse(raw)
problem.get_error_rate()

21996

<IPython.core.display.Javascript object>

## Part Two

In [11]:
RAW2 = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9"""
PROBLEM2 = Problem.parse(RAW2)
PROBLEM2 = PROBLEM2.discard_invalid_tickets()

<IPython.core.display.Javascript object>

In [19]:
def identify_fields(problem: Problem) -> List[str]:
    all_tickets = [problem.my_ticket] + problem.nearby_tickets
    field_names = [rule.name for rule in problem.rules]
    candidates = [
        [
            rule
            for rule in problem.rules
            if all(rule.is_valid(t[i]) for t in all_tickets)
        ]
        for i in range(len(field_names))
    ]

    while True:
        unique_rules = [rule for cand in candidates if len(cand) == 1 for rule in cand]
        if len(unique_rules) == len(field_names):
            return [rule.name for rule in unique_rules]

        for i in range(len(field_names)):
            if len(candidates[i]) > 1:
                cand = [rule for rule in candidates[i] if rule not in unique_rules]
                candidates[i] = cand
    return []

<IPython.core.display.Javascript object>

In [20]:
identify_fields(PROBLEM2)

['row', 'class', 'seat']

<IPython.core.display.Javascript object>

In [27]:
with open("../input/day16.txt") as f:
    raw = f.read().strip()
problem = Problem.parse(raw)
problem = problem.discard_invalid_tickets()
fields = identify_fields(problem)
departure_vals = [
    val for name, val in zip(fields, problem.my_ticket) if name.startswith("departure")
]
assert len(departure_vals) == 6
math.prod(departure_vals)

650080463519

<IPython.core.display.Javascript object>